In [41]:
import numpy as np
import pandas as pd
import alpha_vantage
import time
from alpha_vantage.timeseries import TimeSeries 
from alpha_vantage.techindicators import TechIndicators
from config import config
from alpha_vantage.cryptocurrencies import CryptoCurrencies

In [8]:
#5 calls per minute, 500 per day
ts = TimeSeries(key=config["alpha_vantage"]["key"], output_format = "pandas")
ti = TechIndicators(key=config["alpha_vantage"]["key"], output_format='pandas')

stock_dataframes = {}

for symbol in config["alpha_vantage"]["symbol"]:
    print(symbol)
    time.sleep(60)
    ts_data, ts_meta_data = ts.get_daily_adjusted(symbol, outputsize=config["alpha_vantage"]["outputsize"])
    
    ema_data, ema_meta_data = ti.get_ema(symbol, interval="daily")
    rsi_data, rsi_meta_data = ti.get_rsi(symbol, interval="daily")
    time.sleep(60)
    macd_data, macd_meta_data = ti.get_macd(symbol, interval="daily")
    bband_data, bband_meta_data = ti.get_bbands(symbol, interval="daily")
    ad_data, ad_meta_data = ti.get_ad(symbol, interval="daily")
    
    stock_dataframes[symbol] = pd.concat([ts_data, ema_data, rsi_data, macd_data, bband_data, ad_data], axis=1, join="inner")
    stock_dataframes[symbol].drop(['1. open', '2. high', '3. low', '5. adjusted close', '7. dividend amount', '8. split coefficient'], axis=1, inplace=True)
    stock_dataframes[symbol]

IBM
GOOG
NFLX
AMZN
AAPL
MSFT
TSLA
NVDA
ADBE
INTC


In [12]:
for symbol in stock_dataframes:
    stock_dataframes[symbol].to_csv('StockData/' + symbol + '.csv', index=True)

In [16]:
normalized_stock_dataframes = {}
for symbol in stock_dataframes:
    normalized_stock_dataframes[symbol] = (stock_dataframes[symbol]-stock_dataframes[symbol].min())/(stock_dataframes[symbol].max()-stock_dataframes[symbol].min())
    normalized_stock_dataframes[symbol].to_csv('NormalizedStockData/' + symbol + '.csv', index=True)
    


In [31]:
train_data = {}
test_data = {}
for symbol in normalized_stock_dataframes:
    train_size = int(len(normalized_stock_dataframes[symbol]) * config['data']['train_split_size'])
    test_size = len(normalized_stock_dataframes[symbol]) - train_size
    train_data[symbol] = normalized_stock_dataframes[symbol][test_size:-1]
    test_data[symbol] = normalized_stock_dataframes[symbol][0:test_size]


In [62]:
X_train = {}
y_train = {}
n_past = 30
n_future = 10

for symbol in normalized_stock_dataframes:
    X_train[symbol] = []
    y_train[symbol] = []
    if normalized_stock_dataframes[symbol] is type(stock_dataframes['IBM']):
        normalized_stock_dataframes[symbol] = normalized_stock_dataframes[symbol].to_numpy()
    
for symbol in normalized_stock_dataframes:
    for i in range(n_past, len(normalized_stock_dataframes[symbol]) - n_future + 1):
        X_train[symbol].append(normalized_stock_dataframes[symbol][i - n_past: i, 0: stock_dataframes[symbol].shape[1]])
        y_train[symbol].append(normalized_stock_dataframes[symbol][i + n_future - 1:i + n_future, 0])
    X_train[symbol], y_train[symbol] = np.array(X_train[symbol]), np.array(y_train[symbol])